# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [3]:
city_data= pd.read_csv('output_data/cities.csv')
city_data.head()

,City Name,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tūkrah,32.5341,20.5791,300.71,40,0,2.22,LY,1663318021
1,Russell,32.3502,-85.1999,292.72,100,0,0.00,US,1663317979
2,Mil'kovo,54.7167,158.6167,278.07,87,42,3.25,RU,1663318022
3,Arrondissement de Roanne,46.0000,4.0000,286.45,87,44,2.81,FR,1663318022
4,Thompson,55.7435,-97.8558,277.24,100,0,0.00,CA,1663318023


### Humidity Heatmap

In [6]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [13]:
# Set the locations and weight
locations = city_data[["Latitude","Longtitude"]].astype(float)
humidity = city_data["Humidity"].astype(float)

# Create a Humidity Heatmap
fig = gmaps.figure(center=(10,0.00),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [8]:
hotel_df = city_data.loc[(city_data["Humidity"]<80) &
                         (city_data["Humidity"]>20) &
                             (city_data["Max Temperature"]<35) &
                            (city_data["Max Temperature"]>20) &
                            (city_data["Cloudiness"]<70) &
                            (city_data["Wind Speed"]<5)&
                        (city_data["Wind Speed"]>1)&
                        (city_data["Country"] != "AU")]
len(hotel_df)

0

In [6]:
hotel_df.head()

,City Name,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
8,Veroli,41.6911,13.4177,34.06,34,40,2.57,IT,1659957420
15,Bambous Virieux,-20.3428,57.7575,24.56,64,20,4.12,MU,1659957423
16,Ponta do Sol,32.6667,-17.1000,26.67,72,4,1.53,PT,1659957424
35,Kelo,9.3086,15.8066,29.48,63,61,3.32,TD,1659957432
49,Hilo,19.7297,-155.0900,25.71,74,40,3.60,US,1659957440


### Hotel Map

In [9]:
# Add column for hotel to the DataFrame
hotel_df["Hotel Name"] = ""

# Params dictionary to search for hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Using the the lat/long to identify in which city to look for
for index, row in hotel_df.iterrows():
    #Get each cities lat & long and add to api search
    lat = row["Latitude"]
    lng = row["Longtitude"]
    params["location"] = f"{lat},{lng}"

    # Request url
    hotel_name = requests.get(base_url, params=params)
    hotel_name =  hotel_name.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][1]["name"]
        print("Found hotel for " + hotel_name["results"][0]["name"])
    except (KeyError, IndexError):
        print("Missing result... skipping")

In [8]:
hotel_df.head()

,City Name,Latitude,Longtitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
8,Veroli,41.6911,13.4177,34.06,34,40,2.57,IT,1659957420,Hotel Relais Filonardi
15,Bambous Virieux,-20.3428,57.7575,24.56,64,20,4.12,MU,1659957423,La Case du Pêcheur
16,Ponta do Sol,32.6667,-17.1000,26.67,72,4,1.53,PT,1659957424,Hotel do Campo
35,Kelo,9.3086,15.8066,29.48,63,61,3.32,TD,1659957432,Centro Educativo Charles Lwanga (CECL) - Misio...
49,Hilo,19.7297,-155.0900,25.71,74,40,3.60,US,1659957440,Hilo Hawaiian Hotel
